In [ ]:
USERNAME = 
PASSWORD = 

In [1]:
import os
import time
from datetime import date

import Driver
import pandas as pd
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException

In [2]:
from numpy.random import randint

def waitRand(maxTime=3):
    waitTime = randint(1,maxTime+1)
    time.sleep(waitTime)

In [3]:
import pickle
with open('imageToBuilding.pkl', 'rb') as f:
    imageToBuilding = pickle.load(f)

In [4]:
def find_coords(map_grid):
    """
    map_grid: List of strings, attributes of elements in map table
    Outputs list of x, y coords.
    """
    mapped_map_grid = list(map(lambda x: x.get_attribute('onmouseover').split(','), map_grid))
    filtered_map_grid = list(filter(lambda x: int(x[6]) - int(x[5]) != 0, mapped_map_grid))
    coord_list = list(map(lambda x: [int(x[1]), int(x[2])], filtered_map_grid))
    return coord_list

In [5]:
def extract_building_info(map_tiles):
    def get_raw_info(element):
        img_url = 'https://usa.tycoononline.com/' + element['src']
        building_type = imageToBuilding.get(img_url, None)
        if building_type is None:
            return None
        mouseover_info = element['onmouseover'].split("'")
        company_name = mouseover_info[1]
        player_name = mouseover_info[5]
        road_bonus = mouseover_info[7]
        return [building_type, company_name, player_name, road_bonus]
    
    mapped_map_tiles = list(map(get_raw_info, map_tiles))
    filtered_map_tiles = list(filter(lambda x: x, mapped_map_tiles))
    def fix_my_jank_shit(outer_tuple):
        building_type = outer_tuple[0]
        company_name = outer_tuple[1][3].strip("'")
        player_name = outer_tuple[1][5].strip("'")
        road_bonus = outer_tuple[1][6].strip("'")
        return [building_type, company_name, player_name, road_bonus]
    
    return filtered_map_tiles

In [6]:
driver = Driver.Driver()
driver.login(USERNAME, PASSWORD)

In [7]:
all_buildings = []

for cityId in range(1,11):
    waitRand(maxTime=2)
    cityOverviewUrl = 'https://usa.tycoononline.com/frame_index.php?page=kartaSegment&cityId={}&typ=1'.format(cityId)
    driver.get(cityOverviewUrl)

    map_grid = driver.find_element_by_id("chooseArea").find_element_by_tag_name('table').find_element_by_tag_name('table').find_elements_by_tag_name('td')
    coord_list = find_coords(map_grid)
    print("\nFound coordinates for city {}, total of {} occupied tiles.".format(cityId, len(coord_list)))

    for coord_set in coord_list:
        x = coord_set[0]
        y = coord_set[1]
        cityTileurl = "https://usa.tycoononline.com/frame_index.php?page=kartaTitta&X={}&Y={}&cityId={}".format(x, y, cityId)
        driver.get(cityTileurl)
        full_map = driver.find_element_by_id('main-column').find_elements_by_tag_name('table')[5].get_attribute('innerHTML')
        bs4 = BeautifulSoup(full_map)
        map_tiles = bs4.find_all('img')
        processed_map_tiles = extract_building_info(map_tiles)
        all_buildings += processed_map_tiles
        print("Logged {} buildings for x={}, y={} in city {}.".format(str(len(processed_map_tiles)).zfill(2), str(x).zfill(2), str(y).zfill(2), cityId))


Found coordinates for city 1, total of 36 occupied tiles.
Logged 33 buildings for x=06, y=01 in city 1.
Logged 36 buildings for x=07, y=01 in city 1.
Logged 31 buildings for x=07, y=02 in city 1.
Logged 27 buildings for x=07, y=03 in city 1.
Logged 37 buildings for x=03, y=04 in city 1.
Logged 39 buildings for x=04, y=04 in city 1.
Logged 37 buildings for x=05, y=04 in city 1.
Logged 28 buildings for x=06, y=04 in city 1.
Logged 30 buildings for x=07, y=04 in city 1.
Logged 16 buildings for x=08, y=04 in city 1.
Logged 02 buildings for x=15, y=04 in city 1.
Logged 03 buildings for x=16, y=04 in city 1.
Logged 35 buildings for x=07, y=05 in city 1.
Logged 65 buildings for x=15, y=05 in city 1.
Logged 08 buildings for x=16, y=05 in city 1.
Logged 29 buildings for x=17, y=08 in city 1.
Logged 28 buildings for x=17, y=09 in city 1.
Logged 21 buildings for x=05, y=10 in city 1.
Logged 40 buildings for x=06, y=10 in city 1.
Logged 56 buildings for x=07, y=10 in city 1.
Logged 34 buildings f

Logged 11 buildings for x=16, y=04 in city 7.
Logged 03 buildings for x=01, y=05 in city 7.
Logged 01 buildings for x=15, y=05 in city 7.
Logged 08 buildings for x=16, y=05 in city 7.
Logged 32 buildings for x=17, y=07 in city 7.
Logged 28 buildings for x=17, y=08 in city 7.
Logged 29 buildings for x=17, y=09 in city 7.
Logged 34 buildings for x=15, y=10 in city 7.
Logged 31 buildings for x=16, y=10 in city 7.
Logged 54 buildings for x=17, y=10 in city 7.
Logged 45 buildings for x=18, y=10 in city 7.
Logged 55 buildings for x=06, y=11 in city 7.
Logged 29 buildings for x=15, y=11 in city 7.
Logged 31 buildings for x=16, y=11 in city 7.
Logged 44 buildings for x=06, y=12 in city 7.
Logged 49 buildings for x=07, y=12 in city 7.
Logged 54 buildings for x=08, y=12 in city 7.
Logged 42 buildings for x=09, y=12 in city 7.
Logged 42 buildings for x=07, y=17 in city 7.
Logged 31 buildings for x=08, y=17 in city 7.

Found coordinates for city 8, total of 18 occupied tiles.
Logged 24 buildings f

In [8]:
leaderboard_values = []
currentPage = 0
next_page = 'https://usa.tycoononline.com/frame_index.php?page=rankingCompany&pagenbr=1&sortering=ranking&pageValue1'
while (next_page):
    waitRand()
    driver.get(next_page)
    try:
        next_page = driver.find_element_by_link_text('>').get_attribute('href')
    except NoSuchElementException:
        next_page = None
    leaderboard = driver.find_element_by_xpath(
        "/html/body/div[2]/div[2]/table/tbody/tr/td[2]/table/tbody/tr[2]/td/table[4]")
    rows = leaderboard.find_elements_by_tag_name('tr')
    currentPage += 1
    print('Got page {}.'.format(currentPage))

    for place in range(2, len(rows), 2):
        row = rows[place]
        ranking = int(row.find_elements_by_tag_name('td')[0].text)
        companyName = row.find_elements_by_tag_name('td')[2].text
        playerName = row.find_elements_by_tag_name('td')[3].text
        companyValue = int(row.find_elements_by_tag_name('td')[4].text.replace(',', ''))
        rankingInfo = (ranking, companyName, playerName, companyValue)
        leaderboard_values.append(rankingInfo)

Got page 1.
Got page 2.
Got page 3.
Got page 4.
Got page 5.
Got page 6.
Got page 7.
Got page 8.
Got page 9.
Got page 10.
Got page 11.


In [9]:
driver.close()
print('Done.')

today = date.today()
print('Recorded date of data collection.')

Done.
Recorded date of data collection.


In [10]:
cols = ['Rank', 'Company Name', 'Player Name', 'CV']
leaderboard_df = pd.DataFrame(leaderboard_values, columns=cols).set_index('Rank')

folder = os.path.join("data", "leaderboard")
if not os.path.exists(folder):
    os.makedirs(folder)
path = os.path.join(folder, '{}.csv'.format(today))
leaderboard_df.to_csv(path)

In [11]:
cols = ['Building', 'Company Name', 'Player Name', 'Road Bonus']
buildings_df = pd.DataFrame(all_buildings, columns=cols)

folder = os.path.join("data", "buildingData")
if not os.path.exists(folder):
    os.makedirs(folder)
path = os.path.join(folder, '{}.csv'.format(today))
buildings_df.to_csv(path, index=False)